In [ ]:
import cv2
import numpy as np

In [ ]:
def calculate_t(img, pixel_cnt):
  boolidx = img<=1
  imgtmp = img[boolidx]
  # if imgtmp.shape[0] < pixel_cnt:
  #   thres = np.sum(imgtmp)/imgtmp.shape[0]
  # else:
  #   thres = np.sum(img)/pixel_cnt
  thres = np.sum(img)/pixel_cnt
  return thres

def frequency_grid(img, segment_length):
  w = img.shape[1]
  h = img.shape[0]
  rest_row = h%segment_length
  rest_col = w%segment_length
  segment_size = segment_length**2
  grid_size = (-1*(-w//segment_length), -1*(-h//segment_length))
  grid = np.zeros((grid_size[1], grid_size[0]))
  cur_row = 0
  cur_col = 0
  for i in range(grid_size[1]):
    for j in range(grid_size[0]):
      if i < grid_size[1]-1 and j < grid_size[0]-1:
        img_seg = img[cur_row:cur_row+segment_length, cur_col:cur_col+segment_length]
        grid[i, j] = (img_seg==0).sum()/segment_size
      elif i >= grid_size[1]-1 and j >= grid_size[0]-1:
        img_seg = img[cur_row:, cur_col:]
        if rest_row == 0 and rest_col == 0:
          grid[i, j] = (img_seg==0).sum()/segment_size
        elif rest_col == 0:
          grid[i, j] = (img_seg==0).sum()/(segment_length*rest_row)
        elif rest_row == 0:
          grid[i, j] = (img_seg==0).sum()/(segment_length*rest_col)
        else:
          grid[i, j] = (img_seg==0).sum()/(rest_col*rest_col)
      elif i >= grid_size[1]-1:
        img_seg = img[cur_row:, cur_col:segment_length]
        if rest_row == 0:
          grid[i, j] = (img_seg==0).sum()/segment_size
        else:
          grid[i, j] = (img_seg==0).sum()/(segment_length*rest_row)
      elif j >= grid_size[0]-1:
        img_seg = img[cur_row:segment_length, cur_col:]
        if rest_col == 0:
          grid[i, j] = (img_seg==0).sum()/segment_size
        else:
          grid[i, j] = (img_seg==0).sum()/(segment_length*rest_col)
      
      cur_col += segment_length
    cur_row += segment_length
    cur_col = 0

  return grid

In [ ]:
def global_threshold(img):
  pixel_cnt = img.shape[0]*img.shape[1]
  img_equalized = img.copy()
  prev_thres = 0
  iter_cnt = 0
  while (iter_cnt < 10000):
    thres = calculate_t(img_equalized, pixel_cnt)
    # print('thres: ', thres)
    if prev_thres != 0 and abs(thres - prev_thres) < 0.0001:
      break
    img_subtracted = 1 - thres + img_equalized
    # ret, img_subtracted = cv2.threshold(img_subtracted, 1, 1, cv2.THRESH_TRUNC)
    e_min = np.amin(img_subtracted)
    img_equalized = 1 - ((1-img_subtracted) / (1-e_min))
    prev_thres = thres
    iter_cnt += 1
  # print(iter_cnt)
  # ret, img_equalized = cv2.threshold(img_equalized, 1, 1, cv2.THRESH_TRUNC)
  return img_equalized

def local_threshold(img_ori, seg_len, k):
  img = img_ori.copy()
  freq_grid = frequency_grid(img, seg_len)
  m = np.mean(freq_grid)
  s = np.std(freq_grid)
  grid_w = freq_grid.shape[1]
  grid_h = freq_grid.shape[0]
  cur_row = 0
  cur_col = 0
  for i in range(grid_h):
    for j in range(grid_w):
      if freq_grid[i, j] > m+k*s:
        if j < grid_w-1 and i < grid_h-1:
          img_seg = img[cur_row:cur_row+seg_len, cur_col:cur_col+seg_len]
          img[cur_row:cur_row+seg_len, cur_col:cur_col+seg_len] = global_threshold(img_seg)
        elif j >= grid_w-1 and i >= grid_h-1:
          img_seg = img[cur_row:, cur_col:]
          img[cur_row:, cur_col:] = global_threshold(img_seg)
        elif i >= grid_h-1:
          img_seg = img[cur_row:, cur_col:cur_col+seg_len]
          img[cur_row:, cur_col:cur_col+seg_len] = global_threshold(img_seg)
        elif j >= grid_w-1:
          img_seg = img[cur_row:cur_row+seg_len, cur_col:]
          img[cur_row:cur_row+seg_len:, cur_col:] = global_threshold(img_seg)

      cur_col += seg_len
    cur_row += seg_len
    cur_col = 0
  
  return img

In [ ]:
def IGTthreshold(img):
    img_thres = global_threshold(img)
    seg_len = int(max(img.shape)/5)
    img_thres = local_threshold(img_thres, 50, 2)

    pix_cnt = img.shape[0]*img.shape[1]
    final_thres = calculate_t(img_thres, pix_cnt)
    
    return final_thres, img_thres
